## **Instructions**
* Runtime > Run all
 * select your zip files in below
 * the code should download the merged files

In [ ]:
#@title Processing code

"""
This script merges data of the Empatica E4 into one csv file
"""

__author__      = 'Yong Dich'

import os
import re
import csv
import sys
import functools
import numpy as np
import pandas as pd

from sys import platform
from copy import deepcopy
from itertools import chain 
from functools import reduce
from datetime import datetime

def main(d):
    ''' run this script in the folder of the E4 data'''
    
    # make the path into an absolute path
    if not os.path.isabs(d):
        d = os.path.abspath(d)

    # # add the existing directories to a list
    # directoryList = []
    # if os.path.exists(directory): 
    #     for file in os.listdir(directory): 
    #         groupDirectory = os.path.join(directory, file)
    #         if os.path.isdir(groupDirectory): 
    #             directoryList.append(groupDirectory)
 
    # # go through each directory and process its content
    # fileoutput = []
    # for d in directoryList: 
    print("Processing:", d)
    new_input = []
    input_list = direct(d)
    if len(input_list) > 0: 
        for file in input_list: 
            if file == 'ACC.csv':
                try: 
                    ACC_fun(file, d) 
                    name_of_file = file[:-4] + "_new.csv"
                    new_input.append(name_of_file)
                except Exception as e: 
                    print("issue with the ACC file:", e)
                    continue
            elif file == 'IBI.csv': 
                try: 
                    name_of_file = file[:-4] + "_new.csv"
                    IBI_fun(file, d)
                    new_input.append(name_of_file)
                except: 
                    continue
            elif file == 'tags.csv':
                try: 
                    name_of_file = file[:-4] + "_new.csv"
                    tagEvents(file, d)
                    new_input.append(name_of_file)
                except: 
                    continue 
            else:
                try: 
                    dataFrames(file, d)
                    name_of_file = file[:-4] + "_new.csv"
                    new_input.append(name_of_file)
                except: 
                    continue 
                    
        if len(new_input) > 0: 
            merge(new_input, d)
        else: print("Nothing to merge")
    
    # probably want to merge here or check if there's a merged file
    fileinput = []
    for file in os.listdir(d): 
        if "Merged" in file: 
            #fileoutput.append(os.path.join(d,file))
            return os.path.join(d,file)
        elif "_new" in file: 
            fileinput.append(file)
    if len(fileinput) > 0:
        merge(fileinput, d)
        
# return the merged files
#return fileoutput

def merge(input_list, directory): 
    data_frames = []
    for file in input_list: 
        filename = os.path.join(directory, file)
        df1 = pd.read_table(filename, sep=',')
        data_frames.append(df1)

    df_merged = reduce(lambda left, right: 
                       pd.merge(left, right, on='unix time', how='outer'), data_frames)
    df_merged_sorted = df_merged.sort_values(['unix time'])
    
    # if it's not empty, let's process the tag file
    try:
        tagfile = os.path.join(directory,'tags.csv')  
        df = pd.read_csv(tagfile, header=None)
        tags = df[0][:]
        taglen = len(tags.values)
        finaltagList = []
        for i in range(1, taglen+1): 
            index_tag = df_merged_sorted.index[df_merged_sorted['tags'] == "Tag " + str(i)]
            finaltagList.append((index_tag.item(), "Tag " + str(i)))

        for i in range(len(finaltagList)-1):
            df_merged_sorted.loc[finaltagList[i][0]:finaltagList[i+1][0], 'tags']= finaltagList[i][1]
            if i == len(finaltagList)-2: 
                df_merged_sorted.loc[finaltagList[i+1][0]:df_merged_sorted.last_valid_index(), 'tags']= finaltagList[i+1][1]
            else: continue 
    
    # empty tag file
    except Exception as e: 
        print('Ignoring tag file because ', e, end='; ')

    # merge everything
    findSlashes = (re.findall(r'/(\w+)', directory) or None)
    name = findSlashes[len(findSlashes)-1] + 'Merged.csv'
    df_merged_sorted.to_csv(os.path.join(directory, name), sep=',', na_rep='', index=False)


def direct(directory): 
    input_list = []
    original_length = len(os.listdir(directory))
    acceptablefiles = ["ACC", "BVP", "EDA", "HR", "IBI", "tags", "TEMP"]
    if os.path.exists(directory): 
        for file in os.listdir(directory): 
            checkFile = file[:-4] + "_new.csv"
            if checkFile in os.listdir(directory):
                print("Already have " + checkFile)
                continue
            else: 
                if ('_new' not in file) and file.endswith(".csv") and any(word in file for word in acceptablefiles):
                    input_list.append(file)
                    
    return input_list

def ACC_fun(filename, directory): 
    file = os.path.join(directory, filename) 
    df = pd.read_csv(file, header=None)
    
    initialx = df.loc[0,0]
    initialy = df.loc[0,1]
    initialz = df.loc[0,2]

    numx = df[0][2:]
    numy = df[1][2:]
    numz = df[2][2:]
    hertzx = df[0][1]
    hertzy = df[1][1]
    hertzz = df[2][1]

    # print another
    file_n = filename[:-4]
    final_file_n = file_n + ' real time'
    new_data = {
        'unix time' :[], 
        final_file_n : [], 
        'ACC_x': [], 
        'ACC_y':[], 
        'ACC_z':[]
    }
    sampleNumber = 0
    for item in numx: 
        if (hertzx) or (initialx):
            pass
        # else: 
        new_data['ACC_x'].append(item)
        newcolumn = float(sampleNumber)/hertzx + initialx 
        sampleNumber = sampleNumber + 1
        new_data['unix time'].append(newcolumn)
        realtime = datetime.datetime.fromtimestamp(newcolumn).strftime('%Y-%m-%d %H:%M:%S')
        new_data[final_file_n].append(realtime)
    for item in numy: 
        if (hertzy) or (initialy):
            pass
        # else: 
        new_data['ACC_y'].append(item)
    for item in numz: 
        if (hertzz) or (initialz):
            pass
        # else: 
        new_data['ACC_z'].append(item)

    unix_df = pd.DataFrame(new_data)

    name_of_file = file[:-4] + "_new.csv"
    unix_df.to_csv(os.path.join(directory, name_of_file), 
                   header=True, index=False, encoding='utf-8', mode='a')


def IBI_fun(filename, directory): 
    file = os.path.join(directory, filename)  
    df = pd.read_csv(file, header=None)
    ibi = df[1][1:]
    initial = df[0][0]
    times_from_initial = df[0][1:]
    file_n = filename[:-4]
    final_file_n = file_n + ' real time'
    new_data = {
        'unix time': [], 
        final_file_n: [], 
        file_n: []
    }
    
    for item in ibi: 
        new_data[file_n].append(item)
    
    
    for time in times_from_initial: 
        newunix = initial + time
    
        new_data['unix time'].append(newunix)
        realtime = datetime.datetime.fromtimestamp(newunix).strftime('%Y-%m-%d %H:%M:%S')
        new_data[final_file_n].append(realtime)
    unix_df = pd.DataFrame(new_data)
    name_of_file = file[:-4] + "_new.csv"
    unix_df.to_csv(os.path.join(directory, name_of_file), 
                   header=True, index=False, encoding='utf-8', mode='a')
    
def tagEvents(filename, directory): 
    # read in the file as dataframe
    file = os.path.join(directory, filename)  
    df = pd.read_csv(file, header=None)
    file_n = filename[:-4]
    final_file_n = file_n + ' real time'
    tags = df[0][:]
    new_data = {    
        'unix time': [], 
        file_n: [], 
        final_file_n: []
    }

    # could also have enumerate(tags) for index and values 
    tagNum = 1
    for tag in tags: 
        new_data['unix time'].append(tag)
        new_data[file_n].append("Tag " + str(tagNum))
        realtime = datetime.datetime.fromtimestamp(tag).strftime('%Y-%m-%d %H:%M:%S')
        new_data[final_file_n].append(realtime)
        tagNum += 1
    unix_df = pd.DataFrame(new_data)
    name_of_file = file[:-4] + "_new.csv"

    unix_df.to_csv(os.path.join(directory, name_of_file), 
                   header=True, index=False, encoding='utf-8', mode='a')

def dataFrames(filename, directory):
    file = os.path.join(directory, filename)  
    df = pd.read_csv(file, header=None)
    file_n = filename[:-4]
    final_file_n = file_n + ' real time'

    unixtime = df[0][0]
    freq = df[0][1]
    fileItems = df[0][2:]

    new_data = { 
        'unix time' : [],
        file_n : [], 
        final_file_n : []
    }
    
    sampleNumber = 0
    for item in fileItems: 
        new_data[file_n].append(item)
        newcolumn = float(sampleNumber)/freq + unixtime 
        sampleNumber = sampleNumber + 1
        new_data['unix time'].append(newcolumn)
        realtime = datetime.datetime.fromtimestamp(newcolumn).strftime('%Y-%m-%d %H:%M:%S')
        new_data[final_file_n].append(realtime)

    df = pd.DataFrame(new_data)
    name_of_file = file[:-4] + "_new.csv"
    df.to_csv(os.path.join(directory, name_of_file), 
              header=True, index=False, encoding='utf-8', mode='a')
    


In [ ]:
#@title Upload your zip file
from google.colab import files
import pandas as pd
import datetime
import os

# get the file
content = files.upload()
for filename in list(content.keys()):

  # make sure the filename is unique
  now = str(datetime.datetime.now())
  now = filename + '_' + now
  unique_filename = filename.replace('.csv', '-' + now + '.csv')
  os.rename(filename, unique_filename)

  # unzip the data into a new folder
  ! mkdir "$now"
  ! unzip "$unique_filename" -d "$now"

  # process the data
  output = main(now)
  files.download(output)